---
#Socioeconómico Corrientes (ETL)
---


---
Cargar Dataset
---

In [ ]:
import pandas as pd
import numpy as np


# el parámetro 'header=2' para decirle a Pandas que las cabeceras
# están en la fila con índice 2 (la tercera fila del Excel).
# descartará automáticamente las filas anteriores (0 y 1).

estimacion = pd.read_excel(
    "../Datos Originales/estimacion-por-departamento_y_sector_series_empleo_y_salarios_mensual.xlsx",
    sheet_name="datos",
    header=2
)



# poblacion censo
censo = pd.read_csv("../Datos Originales/Viv_Particulares_Censo2022.csv",delimiter=";",encoding="utf-8",thousands=".")

---
Limpieza dataset
---


🔷estimacion

In [12]:
estimacion.head()

,provincia_id,provincia,departamento_id,departamento,sector,periodo,empleos,salarios
0,2,CIUDAD AUTONOMA DE BUENOS AIRES,2000,Sin dato de comuna,Servicios,201501,s.d.,s.d.
1,2,CIUDAD AUTONOMA DE BUENOS AIRES,2000,Sin dato de comuna,Servicios,201502,s.d.,s.d.
2,2,CIUDAD AUTONOMA DE BUENOS AIRES,2000,Sin dato de comuna,Servicios,201503,s.d.,s.d.
3,2,CIUDAD AUTONOMA DE BUENOS AIRES,2000,Sin dato de comuna,Servicios,201504,s.d.,s.d.
4,2,CIUDAD AUTONOMA DE BUENOS AIRES,2000,Sin dato de comuna,Servicios,201505,s.d.,s.d.


In [ ]:
# filtrar solo para la provincia de Corrientes
df_corrientes = estimacion[estimacion['provincia'] == 'CORRIENTES'].copy()
df_corrientes.head()

,provincia_id,provincia,departamento_id,departamento,sector,periodo,empleos,salarios
114912,18,CORRIENTES,18007,Bella Vista,Servicios,201501,382,7303.593262
114913,18,CORRIENTES,18007,Bella Vista,Servicios,201502,409,7104.765625
114914,18,CORRIENTES,18007,Bella Vista,Servicios,201503,416,7409.928711
114915,18,CORRIENTES,18007,Bella Vista,Servicios,201504,441,7354.838867
114916,18,CORRIENTES,18007,Bella Vista,Servicios,201505,463,7530.930176


In [ ]:
# valores 's.d.' (sin dato)
df_corrientes.replace('s.d.', np.nan, inplace=True)

C:\Users\lucia\AppData\Local\Temp\ipykernel_16880\1476387782.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_corrientes.replace('s.d.', np.nan, inplace=True)


In [11]:
# convertir columnas a tipos de datos numéricos
df_corrientes['empleos'] = pd.to_numeric(df_corrientes['empleos'], errors='coerce')
df_corrientes['salarios'] = pd.to_numeric(df_corrientes['salarios'], errors='coerce')

In [12]:
# convertir la columna 'periodo' a un formato de fecha real
df_corrientes['fecha'] = pd.to_datetime(df_corrientes['periodo'], format='%Y%m')

In [14]:
# convertir la columna 'periodo' a un formato de fecha real
df_corrientes['fecha'] = pd.to_datetime(df_corrientes['periodo'], format='%Y%m')

# nombres de departamentos a mayúsculas
df_corrientes['departamento'] = df_corrientes['departamento'].str.upper()

In [15]:
# renombrar columnas 
df_final = df_corrientes[['fecha', 'departamento', 'sector', 'empleos', 'salarios']].copy()
df_final.rename(columns={'departamento': 'Departamento'}, inplace=True)

In [16]:
df_final.head()

,fecha,Departamento,sector,empleos,salarios
114912,2015-01-01,BELLA VISTA,Servicios,382.0,7303.593262
114913,2015-02-01,BELLA VISTA,Servicios,409.0,7104.765625
114914,2015-03-01,BELLA VISTA,Servicios,416.0,7409.928711
114915,2015-04-01,BELLA VISTA,Servicios,441.0,7354.838867
114916,2015-05-01,BELLA VISTA,Servicios,463.0,7530.930176


In [18]:
df_final.isnull().sum()

fecha              0
Departamento       0
sector             0
empleos         4234
salarios        4234
dtype: int64

In [ ]:
# empleos         4234
# salarios        4234
#valores nulos en las columnas 'empleos' y 'salarios' manejar estos valores nulos en power bi

In [ ]:
# contar el número total de filas duplicadas

numero_duplicados = df_final.duplicated().sum() # <-- .duplicated() marca todas las ocurrencias de un duplicado excepto la primera como True.

print(f"Se encontraron {numero_duplicados} filas duplicadas.")

Se encontraron 0 filas duplicadas.


In [20]:
# guardar el DataFrame final en un archivo CSV
df_final.to_csv('empleos_corrientes_limpio.csv', index=False, encoding='utf-8')

🔷 Población Censo

In [12]:
censo.head()

,Region,Departamento,Municipio,Poblacion_2022
0,RIO SANTA LUCIA,SAN ROQUE,9 DE JULIO,3402.0
1,TIERRA COLORADA,GENERAL ALVEAR,ALVEAR,8722.0
2,RIO SANTA LUCIA,BELLA VISTA,BELLA VISTA,41321.0
3,CENTRO SUR,PASO DE LOS LIBRES,BONPLAND,1369.0
4,NOROESTE,GENERAL PAZ,CAA CATI,8578.0


In [13]:
censo.shape

(77, 4)

In [14]:
censo.isnull().sum()

Region            1
Departamento      1
Municipio         1
Poblacion_2022    1
dtype: int64

In [15]:
# encontrar filas con valores nulos en cualquier columna
censo[censo.isnull().any(axis=1)]

,Region,Departamento,Municipio,Poblacion_2022
76,NaN,NaN,NaN,NaN


In [21]:
# crea un nuevo DataFrame llamado 'censo_limpio' sin las filas que tienen nulos
censo_limpio = censo.dropna().copy()

In [22]:
# verifica si hay valores nulos en el nuevo DataFrame
censo_limpio.isnull().sum()

Region            0
Departamento      0
Municipio         0
Poblacion_2022    0
dtype: int64

In [23]:
# 1. eliminar los puntos de la columna de población.
# 2. convertir la columna a tipo numérico (entero).
censo_limpio['Poblacion_2022'] = censo_limpio['Poblacion_2022'].astype(int)

In [24]:
censo_limpio['Departamento'] = censo_limpio['Departamento'].str.upper()

In [25]:
# guardar el DataFrame limpio en un archivo CSV
censo_limpio.to_csv("Censo_limpio.csv", index=False)